In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.options.display.max_columns = None

In [2]:
# preparing the data to be used in matlab; first pass:
# data has not been checked and could be prepared more carefully

df = pd.read_stata('../../../python/raw_data/dataprep2.dta')

cohort_list = ['cohort' + str(x) for x in range(1,8)]
time_list = ['time' + str(x) for x in range(1,8)]
df2 = df.loc[df['realyear'] > 94, # asset data for 1994 is underreported and hence not used
         ['HHID', 'age','male', 'PI', 'heal', 'assets', 'hhinc', 'medcost', 'realyear'] \
             + time_list+ cohort_list].copy()

# convert time1-time7 and cohort1-cohort6 to time and cohort vars
def get_dum(row, col_names):  
    for c in col_names:
        if row[c] == 1:
            return int(c[-1])
df2['time'] = df2.loc[:, time_list].apply(get_dum, args = (time_list,), axis=1)
df2['cohort'] = df2.loc[:, cohort_list].apply(get_dum, args = (cohort_list,), axis=1)
df2.drop(time_list+cohort_list, axis=1, inplace=True)

df2['PI'] = df2.groupby('HHID')['PI'].transform('mean')
df2.rename(inplace=True, 
           columns={'male': 'g', 'heal': 'h', 'assets': 'a', 
                    'hhinc': 'inc', 'PI': 'I', 'medcost': 'med'})
bins = np.linspace(0, 1, num=6, endpoint=True)
names = [x for x in range(1,6)]
df2['quintile'] = pd.cut(df2['I'], bins, labels=names)
df2['cohort'] = df2['cohort'].replace({7:6}) -1 # group last category and start at 1

df3 = df2.loc[df2.realyear == 96,].dropna(subset=['a', 'med'])

df3 = df3[[ 'g', 'I', 'h', 'a', 'inc', 'med', 'age', 'quintile', 'time', 'cohort']]
df3 = df3[df3['age']<100]

# prepare data for use in matlab

df3.to_csv('../data/data_init.csv')

In [5]:
cohort1 = df2.loc[df2['age'].between(74,84) & (df2['g'] == 0) & (df2['cohort'] == 1),] \
        . groupby(['quintile','age']) \
        .agg({'a': 'median'}).unstack('quintile')
cohort1all = df2.loc[df2['age'].between(74, 84) & (df2['g'] == 0) & (df2['cohort'] == 1),] \
        . groupby(['age']) \
        .agg({'a': 'median'})
cohort1 = pd.concat([cohort1, cohort1all], axis= 1)
cohort1.columns = ['q' + str(i) for i in range(1,6)] + ['all']
cohort1.to_csv('../data/cohort1_female.csv')

cohort1 = df2.loc[df2['age'].between(74,84) & (df2['g'] == 1) & (df2['cohort'] == 1),] \
        . groupby(['quintile','age']) \
        .agg({'a': 'median'}).unstack('quintile')
cohort1all = df2.loc[df2['age'].between(74, 84) & (df2['g'] == 1) & (df2['cohort'] == 1),] \
        . groupby(['age']) \
        .agg({'a': 'median'})
cohort1 = pd.concat([cohort1, cohort1all], axis= 1)
cohort1.columns = ['q' + str(i) for i in range(1,6)] + ['all']
cohort1.to_csv('../data/cohort1_male.csv')

In [6]:
cohort2 = df2.loc[df2['age'].between(84, 94) & (df2['g'] == 0) & (df2['cohort'] == 2),] \
        . groupby(['quintile','age']) \
        .agg({'a': 'median'}).unstack('quintile')
cohort2all = df2.loc[df2['age'].between(84, 94) & (df2['g'] == 0) & (df2['cohort'] == 2),] \
        . groupby(['age']) \
        .agg({'a': 'median'})
cohort2 = pd.concat([cohort2, cohort2all], axis= 1)
cohort2.columns = ['q' + str(i) for i in range(1,6)] + ['all']
cohort2.to_csv('cohort2.csv')